In [ ]:
import copy
import spacy
nlp_en = spacy.load('en')
nlp_de = spacy.load('de')
from spacy.en import English
parser = English()

In [ ]:
from spacy.en import English
parser = English()

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
def create_prompt_response_map(gram):
    prompt_response_map = {}
    for prompt_unit in gram:
        key = None
        responses = []
        for prompt in prompt_unit:
            if prompt.tag == None:
                print("None")
            else:
                if(prompt.tag == 'prompt'):
                    key = prompt.text
                elif(prompt.tag == 'response'):
                    responses.append(prompt.text)
        prompt_response_map[key] = responses
    return prompt_response_map

def read_grammar_and_create_map(file):
    tree = ET.parse(file)
    grammar = tree.getroot()
    return create_prompt_response_map(grammar)

def get_sag_prompts(prompt_response_map):
    sag_prompts = {}
    for key in prompt_response_map:
        if "Sag" in key:
            sag_prompts[key] = prompt_response_map[key]
    return sag_prompts

    
grammar = read_grammar_and_create_map('referenceGrammar.xml')
#sag_prompts = get_sag_prompts(prompt_response_map)

In [ ]:
test_file_csv = "textProcessing_testKaldi.csv"
test_data = None
with open(test_file_csv, 'r') as reader:
    test_data = reader.readlines()  
    
def prompt_map_test_data():
    prompt_map = {}
    for item in test_data[1:]:
        split = item.replace("\n", "").split("\t")
        prompt = split[1]
        
        if prompt in prompt_map:
            prompt_map[prompt].append({'id': split[0], "transcript": split[3], "prompt": split[1]})
        else:
            arr = []
            arr.append({'id':split[0], "transcript":split[3], "prompt": split[1]})
            prompt_map[prompt] = arr
            
    return prompt_map
prompt_test_map = prompt_map_test_data()

# Create Basis based on Reference Grammar and Normal Case

In [ ]:
special_case_map = {"don't": "do not",
 "haven't" : "have not",
 "i'd" : "i would",
 "i'm" : "i am",
 "isn't": "is not",
 "it's": "it is",
 "o'clock": "o'clock",
 "that's" :"that is",
 "there's": "there is"}

def sentence_to_normal_case(sentence):
    trans_sentence = ""
    words = sentence.split(" ")
    for word in words:
        if word in special_case_map:
            word = special_case_map[word]
        trans_sentence += word + " "
    
    return trans_sentence[:-1]

meaning_false = {}
for prompt_unit in prompt_test_map:
    for response in prompt_test_map[prompt_unit]:
        
        sentence = response['transcript']
        if "***" in sentence:
            continue
        if "'" in sentence:
            sentence = sentence_to_normal_case(sentence)
            #print(dict_prompt['transcript'])
            
        sentence = ' '.join(sentence.split())
        if sentence not in prompt_response_map[prompt_unit]:
            meaning_false[response['id']] = {"sentence":sentence, "prompt":response['prompt']}

In [ ]:
prompt_meaning_false = {}
for id_ in meaning_false:
    for item in meaning_false[id_]:
        prompt_key = meaning_false[id_]["prompt"]
        if prompt_key in prompt_meaning_false:
            prompt_meaning_false[prompt_key].append(meaning_false[id_]["sentence"])
        else:
            arr = []
            arr.append(meaning_false[id_]["sentence"])
            prompt_meaning_false[prompt_key] = arr


# Extract Meaning Words

In [ ]:
def nlp_sentence(sentence):
    parsed = parser(sentence)
    lemmas = []
    words = []
    tags = []
    poss = []
    for i, token in enumerate(parsed):
        lemma = token.lemma_
        words.append(token)
        lemmas.append(lemma)
        tag = token.tag_
        pos = token.tag_
        tags.append(tag)
        poss.append(pos)
    return words, lemmas, tags, poss

In [ ]:
len(meaning_false)

In [ ]:
test_key = "Sag: Ich habe keine Reservation"
prompt_meaning_false[test_key]

In [ ]:
def replace_sentence_emp_words(sentence):
    return sentence.replace("thank you ", "")\
    .replace(" please", "")\
    .replace("yes ", "")\
    .replace("thank you ", "")\
    .replace("no ", "")

In [ ]:
def spacy_words_to_string_array(words):
    arr = []
    for word in words:
        arr.append(str(word))
    return arr

In [ ]:
def extract_key_nouns(nlp_sent):
    tags = nlp_sent[2]
    words = spacy_words_to_string_array(nlp_sent[0])
    sentence = " ".join(words) + " "
    #words = " ".join(nlp_sent[0])

    tags_string = " ".join(tags)
    patterns = [['CD', 'JJ', 'NN'], ['JJ', 'NN'], ['DT', 'NN'], ['CD', 'JJ', 'NNS'], ['JJ', 'NNS'], ['DT', 'NNS']]
    extracted_words = []
    for pattern in patterns:
        pattern_string = " ".join(pattern) + " "
        if pattern_string in tags_string: 
            if pattern[-1] not in tags and pattern[0] not in tags:
                continue
            pattern_start_index = tags.index(pattern[0])
            pattern_end_index = tags.index(pattern[-1])
            if pattern_end_index - pattern_start_index + 1 == len(pattern):
                extracted_part = ""
                for i in range(pattern_start_index, pattern_end_index+1):
                    extracted_part += words[i] + " "
                
                extracted_words.append(extracted_part[:-1])
    return extracted_words

#print(nlp_sentence("i am looking for one musical ticket")[2])    
extract_key_nouns(nlp_sentence("i am looking for one musical ticket"))

In [ ]:
counter = 0
result = []
for sentence in grammar[test_key]:
    replaced_sentence = replace_sentence_emp_words(sentence)
    nlp_sent = nlp_sentence(replaced_sentence)
    tags = nlp_sent[2]
    if tags[0] == 'PRP':
        counter +=1
        result.extend(extract_key_nouns(nlp_sent))
        print(str(spacy_words_to_string_array(nlp_sent[0])))
        print(nlp_sent[2])
print(counter)
print(len(result))
print(set(result))

In [ ]:
def extract_key_verbs_by_prp(nlp_sent):
    tags = nlp_sent[2]
    words = spacy_words_to_string_array(nlp_sent[0])
    sentence = " ".join(words)
    #words = " ".join(nlp_sent[0])

    tags_string = " ".join(tags)
    prio_one_patterns = [['VBP', 'VBG', 'IN'], ['MD', 'VB', 'CD'], ['VBP', 'RB']]
    # Only when no prio_one_pattern fits
    prio_sec_patterns = [['VBP']]
    match = False
    extracted_words = []
    for pattern in prio_one_patterns:
        pattern_string = " ".join(pattern)
        if pattern_string in tags_string:  
            pattern_start_index = tags.index(pattern[0])
            pattern_end_index = tags.index(pattern[-1])
            if pattern_end_index - pattern_start_index + 1 == len(pattern):
                extracted_part = ""
                for i in range(pattern_start_index, pattern_end_index+1):
                    extracted_part += words[i] + " "
                
                extracted_words.append(extracted_part[:-1])
    if len(extracted_words) <1:
        for pattern in prio_sec_patterns:
            pattern_string = " ".join(pattern)
            if pattern_string in tags_string:  
                pattern_start_index = tags.index(pattern[0])
                pattern_end_index = tags.index(pattern[-1])
                if pattern_end_index - pattern_start_index + 1 == len(pattern):
                    extracted_part = ""
                    for i in range(pattern_start_index, pattern_end_index+1):
                        extracted_part += words[i] + " "

                    extracted_words.append(extracted_part[:-1])
        
    return extracted_words

extract_key_verbs_by_prp(nlp_sentence("i am looking for one musical ticket"))

In [ ]:
def extract_key_questions(nlp_sent):
    tags = nlp_sent[2]
    words = spacy_words_to_string_array(nlp_sent[0])
    sentence = " ".join(words)
    #words = " ".join(nlp_sent[0])

    tags_string = " ".join(tags)
    prio_one_patterns = [['MD', 'PRP', 'VB'], ['VBP', 'PRP', 'VBN'], ['VBP', 'PRP', 'VB']]
    # Only when no prio_one_pattern fits
    #prio_sec_patterns = [['VBP']]
    match = False
    extracted_words = []
    for pattern in prio_one_patterns:
        pattern_string = " ".join(pattern) + " "
        if pattern_string in tags_string:
            if pattern[0] not in tags and pattern[-1] not in tags:
                continue
            pattern_start_index = tags.index(pattern[0])
            pattern_end_index = tags.index(pattern[-1])
            if pattern_end_index - pattern_start_index + 1 == len(pattern):
                extracted_part = ""
                for i in range(pattern_start_index, pattern_end_index+1):
                    extracted_part += words[i] + " "
                
                extracted_words.append(extracted_part[:-1])
        
    return extracted_words

_test_key = "have you got one musical ticket"
#print(nlp_sentence(_test_key)[2])
extract_key_questions(nlp_sentence(_test_key))


In [ ]:
counter = 0
result_prp = []
result_md = []
for sentence in grammar[test_key]:
    replaced_sentence = replace_sentence_emp_words(sentence)
    nlp_sent = nlp_sentence(replaced_sentence)
    tags = nlp_sent[2]
    if tags[0] != 'PRP':
        extracted = extract_key_questions(nlp_sent)
        if len(extracted) < 1:
            print(sentence)
            print(nlp_sent[2])
            #print(extract_key_verbs_by_prp(nlp_sent))
        else:
            result_md.extend(extracted)
            

        #print(str(spacy_words_to_string_array(nlp_sent[0])))
        #print(nlp_sent[2])
print(counter)
print(len(result_prp))
print(set(result_md))

# Combine NN, Q, V

In [ ]:
extract_key_nouns(nlp_sentence("i do not have a reservation"))

In [ ]:
for sent in errors_nouns:
    n_s = nlp_sentence(sent)
    try:    
        extract_key_nouns(n_s)
    except:
        print(sent)
        print(n_s)
    

In [ ]:
verbs = []
nouns = []
errors_nouns = []
errors_verbs = []
for key in grammar:
    for response in grammar[key]:
        replaced_sentence = replace_sentence_emp_words(response)
        nlp_sent = nlp_sentence(replaced_sentence)  
        tags = nlp_sent[2]
        if tags[0] == 'PRP':
            try:
                verbs.extend(extract_key_verbs_by_prp(nlp_sent))
            except:
                errors_verbs.append(response)
            try:    
                nouns.extend(extract_key_nouns(nlp_sent))
            except:
                errors_nouns.append(response)
            
        
print(len(set(verbs)))
print(len(set(nouns)))

In [ ]:
for verb in set(verbs):
    print(verb)

In [ ]:
verbs = []
nouns = []
for response in grammar[test_key]:
    replaced_sentence = replace_sentence_emp_words(response)
    nlp_sent = nlp_sentence(replaced_sentence)  
    tags = nlp_sent[2]
    if tags[0] == 'PRP':
        verbs.extend(extract_key_verbs_by_prp(nlp_sent))
        nouns.extend(extract_key_nouns(nlp_sent))
        
        
print(set(verbs))
print(set(nouns))

In [ ]:
for item in prompt_test_map[test_key]:
    if item['transcript'] not in grammar[test_key]:
        print(item['transcript'])

In [ ]:
sag_prompts = get_sag_prompts(grammar)
for key in sag_prompts:
    print(key)

# Cluster approach

In [ ]:
test_prompts = ["Sag: Ich möchte mit Dollars bezahlen",
"Sag: Ich möchte mit Euros bezahlen",
"Sag: Ich möchte mit Kreditkarte bezahlen",
"Sag: Ich möchte mit Mastercard bezahlen",
"Sag: Ich möchte mit Postkarte bezahlen",
"Sag: Ich möchte mit Visa bezahlen",
"Sag: Ich möchte mit Kreditkarte bezahlen",
"Sag: Ich möchte mit Mastercard bezahlen",
"Sag: Ich möchte mit Postkarte bezahlen",
"Sag: Ich möchte mit Visa bezahlen",
"Sag: Ich möchte mit Dollars bezahlen",
"Sag: Ich möchte mit Euros bezahlen",
"Sag: Ich möchte mit Kreditkarte bezahlen",
"Sag: Ich möchte mit Mastercard bezahlen",
"Sag: Ich möchte mit Postkarte bezahlen",
"Sag: Ich möchte mit Visa bezahlen"]

In [ ]:
test_prompts_sentences = []
counter_max = 0
max_prompt = ""
for test_prompt in test_prompts:
    tp_prompts = grammar[test_prompt]
    if len(tp_prompts) > counter_max:
        max_prompt = test_prompt
        counter_max = len(tp_prompts)
        
    test_prompts_sentences.extend(tp_prompts)
print(len(test_prompts_sentences))
print(counter_max)
print(max_prompt)

In [278]:
def is_slice_in_list(s,l):
    len_s = len(s) #so we don't recompute length of s on every iteration
    return any(s == l[i:len_s+i] for i in range(len(l) - len_s+1))

def extract_by_pattern(patterns, tags, words):
    extracted_words = []
    tags_string = " ".join(tags)
    for pattern in patterns:
        pattern_string = " ".join(pattern)
        if is_slice_in_list(pattern, tags):
            pattern_start_index = tags.index(pattern[0])
            if len(pattern) > 1:
                pattern_end_index = tags.index(pattern[-1], pattern_start_index+1)
                if pattern_end_index - pattern_start_index + 1 == len(pattern):
                    extracted_part = ""
                    for i in range(pattern_start_index, pattern_end_index+1):
                        extracted_part += words[i] + " "

                    extracted_words.append(extracted_part[:-1])
            else:
                extracted_words.append(words[pattern_start_index])
    return extracted_words
    
def extract_key_nouns_credit_card(nlp_sent):
    tags = nlp_sent[2]
    words = spacy_words_to_string_array(nlp_sent[0])
    sentence = " ".join(words)
    #words = " ".join(nlp_sent[0])

    prio_one_patterns = [['NN', 'NNS'], ['NN', 'NN']]
    
    # Only when no prio_one_pattern fits
    prio_sec_patterns = [['NN'], ['NNS']]
    match = False
    extracted_words = extract_by_pattern(prio_one_patterns, tags, words)
    if len(extracted_words) < 1:
        extracted_words = extract_by_pattern(prio_sec_patterns, tags, words)
    
    return extracted_words

In [279]:
testtt = "can i pay by credit card"
print(nlp_sentence(testtt))
extract_key_nouns_credit_card(nlp_sentence(testtt))

([can, i, pay, by, credit, card], ['can', 'i', 'pay', 'by', 'credit', 'card'], ['MD', 'PRP', 'VB', 'RP', 'NN', 'NN'], ['MD', 'PRP', 'VB', 'RP', 'NN', 'NN'])


['credit card']

In [284]:
test_key = "Sag: Ich möchte mit Kreditkarte bezahlen"

nouns = []
for key in test_prompts:
    print(key)
    for item in grammar[key]:
        nlp_se = nlp_sentence(item)
        nouns.extend(extract_key_nouns_credit_card(nlp_se))
print(set(nouns))

Sag: Ich möchte mit Dollars bezahlen
Sag: Ich möchte mit Euros bezahlen
Sag: Ich möchte mit Kreditkarte bezahlen
Sag: Ich möchte mit Mastercard bezahlen
Sag: Ich möchte mit Postkarte bezahlen
Sag: Ich möchte mit Visa bezahlen
Sag: Ich möchte mit Kreditkarte bezahlen
Sag: Ich möchte mit Mastercard bezahlen
Sag: Ich möchte mit Postkarte bezahlen
Sag: Ich möchte mit Visa bezahlen
Sag: Ich möchte mit Dollars bezahlen
Sag: Ich möchte mit Euros bezahlen
Sag: Ich möchte mit Kreditkarte bezahlen
Sag: Ich möchte mit Mastercard bezahlen
Sag: Ich möchte mit Postkarte bezahlen
Sag: Ich möchte mit Visa bezahlen
{'card', 'credit card', 'mastercard', 'visa', 'dollars', 'cards', 'master card', 'euros', 'credit cards', 'post card'}
